# Hadoop Streaming assignment 1: Words Rating

The purpose of this task is to create your own WordCount program for Wikipedia dump processing and learn basic concepts of the MapReduce.

In this task you have to find the 7th word by popularity and its quantity in the reverse order (most popular first) in Wikipedia data (`/data/wiki/en_articles_part`).

There are several points for this task:

1) As an output, you have to get the 7th word and its quantity separated by a tab character.

2) You must use the second job to obtain a totally ordered result.

3) Do not forget to redirect all trash and output to /dev/null.

Here you can find the draft of the task main steps. You can use other methods for solution obtaining.

## Step 1. Create mapper and reducer.

<b>Hint:</b>  Demo task contains almost all the necessary pieces to complete this assignment. You may use the demo to implement the first MapReduce Job.

In [1]:
%%writefile mapper1.py
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper1.py


In [2]:
%%writefile reducer1.py
import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Overwriting reducer1.py


## Step 2. Create sort job.

<b>Hint:</b> You may use MapReduce comparator to solve this step. Make sure that the keys are sorted in ascending order.

In [3]:
%%writefile mapper2.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
        
    print "%s\t%d" % (word, count)

Overwriting mapper2.py


## Step 3. Bash commands

<b> Hint: </b> For printing the exact row you may use basic UNIX commands. For instance, sed/head/tail/... (if you know other commands, you can use them).

To run both jobs, you must use two consecutive yarn-commands. Remember that the input for the second job is the ouput for the first job.

In [4]:
%%bash

OUT_DIR="wordcount_result_1"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming WordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null


OUT_DIR_2="wordcount_result_2"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR_2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapred.text.key.comparator.options="-k2,2nr" \
    -D stream.num.map.output.key.fields=2 \
    -D mapred.jab.name="Streaming wordCount 2" \
    -D mapreduce.job.reduces=1 \
    -files mapper2.py \
    -mapper "python mapper2.py" \
    -reducer "python mapper2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR_2} > /dev/null

hdfs dfs -cat ${OUT_DIR_2}/part-00000 | head -7 | tail -1

is	126420


rm: `wordcount_result_1': No such file or directory
19/02/11 18:25:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/11 18:25:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/11 18:25:37 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/11 18:25:38 INFO mapreduce.JobSubmitter: number of splits:2
19/02/11 18:25:38 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549909143504_0001
19/02/11 18:25:38 INFO impl.YarnClientImpl: Submitted application application_1549909143504_0001
19/02/11 18:25:38 INFO mapreduce.Job: The url to track the job: http://1081cd87ddbe:8088/proxy/application_1549909143504_0001/
19/02/11 18:25:38 INFO mapreduce.Job: Running job: job_1549909143504_0001
19/02/11 18:25:46 INFO mapreduce.Job: Job job_1549909143504_0001 running in uber mode : false
19/02/11 18:25:46 INFO mapreduce.Job:  map 0% reduce 0%
19/02/11 18:26:05 INFO mapreduce.Job:  map 47% reduce 0%
19/02/11 18:26:17 INFO mapre